In [1]:
## DESCRIÇÃO AQUI - DEQ 4 

In [2]:
# Imports
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# Definindo a lista de ativos que compõem o índice IBXX a partir de arquivo excel
arquivo = pd.read_excel("ibxx.xlsx")
ibxx = arquivo['Código'].sort_values().tolist() #ordem alfabética, transforma em lista
for i in range(len(ibxx)): #Adiciona '.SA' a cada ativo, para trabalhar com yfinance
    ibxx[i] = ibxx[i]+".SA"
# Carrega os dados de fechamentos de todos os ativos que compõesm o índice
start = '1995-01-01'
end = datetime.today() #Hoje#
#cria um dataframe com os preços de fechamento dos ativos da lista ibxx:
df = yf.download(tickers=ibxx, start=start, end=end).copy()["Adj Close"].round(decimals=2)

[*********************100%***********************]  99 of 99 completed


In [4]:
# next step: screening mostrando a lista de ativos que se enquadram no modelo
lista_enquadrados = []
def screening():
    # Localizar quatro dias em queda:
    for coluna in df.columns:
        #Seleciona as últimas 4 linhas da coluna
        ultimas_6_linhas = df[coluna].tail(6)
        #Verifica se as últimas 4 linhas estão em ordem decrescente e se a quinta é maior que a sexta
        cond1 = (ultimas_6_linhas.iloc[-1] < ultimas_6_linhas.iloc[-2]).all() and \
            (ultimas_6_linhas.iloc[-2] < ultimas_6_linhas.iloc[-3]).all() and \
            (ultimas_6_linhas.iloc[-3] < ultimas_6_linhas.iloc[-4]).all() and \
            (ultimas_6_linhas.iloc[-4] < ultimas_6_linhas.iloc[-5]).all()
        cond2 = (ultimas_6_linhas.iloc[-5] > ultimas_6_linhas.iloc[-6]).all()

        quatro_em_queda = cond1 and cond2
        if quatro_em_queda:
            lista_enquadrados.append(coluna)
    return lista_enquadrados
lista_enquadrados = screening()


In [5]:
# next step: isolar os dados dos ativos contidos na lista e padronizar o formato de saida dos dataframes
df_dict = {}
for coluna in lista_enquadrados:
    df_coluna = pd.DataFrame()
    df_coluna['Close'] = df[coluna]
    df_dict[coluna] = df_coluna
for coluna, df in df_dict.items():
    df.columns.name = coluna
    
df_dict


{}

In [6]:
#Função Backtest individual - Input: df, Output: df_result
def apply_bt(df):
    # Adiciona os preços deslocados de um em n dias, para trás, para calcular os retornos
    df['1d'] = df['Close'].shift(-1)
    df['2d'] = df['Close'].shift(-2)
    df['3d'] = df['Close'].shift(-3)
    df['4d'] = df['Close'].shift(-4)
    df['5d'] = df['Close'].shift(-5)
    df['6d'] = df['Close'].shift(-6)
    df['7d'] = df['Close'].shift(-7)
    df['8d'] = df['Close'].shift(-8)
    df['9d'] = df['Close'].shift(-9)
    df['10d'] = df['Close'].shift(-10)

    # Adiciona a coluna "sinal"
    # Percorre todas as linhas do dataframe e valida o sinal
    for i in range(5, len(df)):
        # Seleciona as últimas 6 linhas da coluna
        ultimas_6_linhas = df.iloc[i-5:i+1, 0]
        # Verifica se as últimas 4 linhas estão em ordem decrescente e se a quinta é maior que a sexta
        cond1 = (ultimas_6_linhas.iloc[-1] < ultimas_6_linhas.iloc[-2]).all() and \
                (ultimas_6_linhas.iloc[-2] < ultimas_6_linhas.iloc[-3]).all() and \
                (ultimas_6_linhas.iloc[-3] < ultimas_6_linhas.iloc[-4]).all() and \
                (ultimas_6_linhas.iloc[-4] < ultimas_6_linhas.iloc[-5]).all()
        cond2 = (ultimas_6_linhas.iloc[-5] > ultimas_6_linhas.iloc[-6]).all()
        quatro_em_queda = cond1 and cond2
        # Atribui 1 para a coluna 'sinal' se a condição for válida, 0 se não
        if quatro_em_queda:
            df.loc[df.index[i], 'sinal'] = 1
        else:
            df.loc[df.index[i], 'sinal'] = 0

    # Remove as linhas onde há valores NaN e 0
    df = df.dropna()
    df = df.drop(df[df['sinal'] == 0].index)

    # Cria df com o cálculo dos retornos
    df_ret = pd.DataFrame()
    df_ret['Close'] = df['Close']
    df_ret['1'] = (((df['1d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['2'] = (((df['2d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['3'] = (((df['3d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['4'] = (((df['4d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['5'] = (((df['5d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['6'] = (((df['6d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['7'] = (((df['7d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['8'] = (((df['8d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['9'] = (((df['9d']/df['Close'])-1)*100).round(decimals=2)
    df_ret['10'] = (((df['10d']/df['Close'])-1)*100).round(decimals=2)
    df_ret

    # Cria df com os resultados
    df_result = pd.DataFrame(index=['Total ocorrências', 'Média +', 'Maior', 'Média -', 'Menor', 'Taxa% +', 'Taxa% -', 'Exp.Mat%'])
    # Cria colunas para cada período
    for i in range(1, 11):
        periodo_coluna = str(i)
        df_result[periodo_coluna] = pd.Series()
    #Adiciona valores total operações
    total_ops = df_ret.shape[0] #número de linhas no df já filtrado com os sinais válidos
    df_result.loc['Total ocorrências'] = total_ops
    df_result = df_result.fillna('')    

    # Função calcula accuracy%
    def acc_pct (coluna):
        return((((coluna > 0).sum())/len(coluna))*100).round(decimals=2)
    # Função calcula média +
    def media_mais(df, coluna):
        valores_positivos = df[df[coluna] > 0][coluna]
        soma_positivos = valores_positivos.sum()
        media_soma_positivos = soma_positivos / len(valores_positivos)
        return media_soma_positivos.round(decimals=2)
    # Função calcula média -
    def media_menos(df, coluna):
        valores_negativos = df[df[coluna] < 0][coluna]
        soma_negativos = valores_negativos.sum()
        media_soma_negativos = soma_negativos / len(valores_negativos)
        return media_soma_negativos.round(decimals=2)
    # Função Maior valor
    def encontrar_maior_valor(df, coluna):
        return df_ret[coluna].max()
    # Função Menor Valor
    def encontrar_menor_valor(df, coluna):
        return df_ret[coluna].min()
    # Função Exp Matemática
    def calc_exp(df, coluna):
        exp = (df_ret[coluna].mean())
        return exp

    # Atribuindo os valores no df:
    for i in df_result.columns:
        df_result.loc['Média +', i] = media_mais(df_ret, i)
    for i in df_result.columns:
        df_result.loc['Média -', i] = media_menos(df_ret, i)

    for i in df_result.columns:
        df_result.loc['Taxa% +', i] = acc_pct(df_ret[i])
    for i in df_result.columns:
        df_result.loc['Taxa% -', i] = 100 - acc_pct(df_ret[i])

    for i in df_result.columns:
        df_result.loc['Maior', i] = encontrar_maior_valor(df_ret[i], i)
    for i in df_result.columns:
        df_result.loc['Menor', i] = encontrar_menor_valor(df_ret[i], i)

    for i in df_result.columns:
        df_result.loc['Exp.Mat%', i] = calc_exp(df_ret[i], i)
    
    df_result.rename_axis(df.columns.name, axis=1, inplace=True) #aplica o nome do eixo das colunas
    pd.set_option('display.width', 200) #formato exibido - padrão=80
    return df_result, print(df_result), print('-' * len(df_result.columns))
   

In [7]:
#Exibe quais ativos se enquadram na condição:
if lista_enquadrados == []:
    print("Nenhum ativo se enquadra na condição em {}".format(df.index[-1].strftime('%Y-%m-%d')))
else:
    print("Ativos em queda por 4 dias consecutivos em {}: ".format(df.index[-1].strftime('%Y-%m-%d')))
    print(lista_enquadrados)
    
#Aplica bt individual para cada componente do df_dict (df dos ativos enquadrados)
for chave in df_dict:
    apply_bt(df_dict[chave])
    

Nenhum ativo se enquadra na condição em 2023-08-25
